<a href="https://colab.research.google.com/github/tanhhkhcndn/mobilenet-transfer-learning/blob/main/mobilenetSSD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training MobileNetSSD Object Detection on a Custom Dataset
## Overview
This notebook walks through how to train a MobileNet object detection model using the TensorFlow 1.5 Object Detection API.

In this specific example, we'll training an model to detect thief in edge devices.
## **Our Data**

We'll be using an image dataset from videos. Our dataset contains 614 images.
We create two datasets in Roboflow: `train`, `test` and `valid`. Use Roboflow to generate TFRecords for each, replace their URLs in this notebook, and you're able to train on your own custom dataset.

## **Our Model**

We'll be training a MobileNetSSDv2 (single shot detector). This specific model is a one-short learner, meaning each image only passes through the network once to make a prediction, which allows the architecture to be very performant for low performace hardware.

The model arechitecture is one of many available via TensorFlow's [model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md#coco-trained-models).

As a note, this notebook presumes TensorFlow 1.5 as TensorFlow 2.0 has yet to fully support the object detection API.

## **Training**

Google Colab provides free GPU resources. Click "Runtime" → "Change runtime type" → Hardware Accelerator dropdown to "GPU."

Colab does have memory limitations, and notebooks must be open in your browser to run. Sessions automatically clear themselves after 12 hours.


## Step 1: Clone TensorFlow2 object detection API folder to your colab:

In [12]:
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 4065, done.
remote: Counting objects: 100% (4065/4065), done.
remote: Compressing objects: 100% (3089/3089), done.
remote: Total 4065 (delta 1186), reused 1941 (delta 916), pack-reused 0
Receiving objects: 100% (4065/4065), 54.71 MiB | 28.60 MiB/s, done.
Resolving deltas: 100% (1186/1186), done.


## Step 2: Install object detection API in colab

In [ ]:
# Install the Object Detection API
%%bash
cd /content/models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

## Step 3: Import the TensorFlow libraries to colab

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

import os
import random
import io
import imageio
import glob
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import colab_utils
from object_detection.builders import model_builder

%matplotlib inline

## Step 4: Run this command to test if object detection API is installed properly:

In [ ]:
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

## Step 5: Prepare custom set of images for training. I’m using Roboflow in this example to create the TFRecord file from the training images.
Downloaded files will go to /content/train/ folder in colab environment.

In [1]:
%cd /content

#Download Training set from git by cloning rep:
import os
import pathlib
# Clone the training set repository if it doesn't already exist
if "ThiefTFData" in pathlib.Path.cwd().parts:
  while "ThiefTFData" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('ThiefTFData').exists():
  !git clone --depth 1 https://github.com/tanhhkhcndn/mobilenet-transfer-learning/
  %cd /content/mobilenet-transfer-learning/ThiefTFData
#  !unzip Pickachu.v1.tfrecord.zip -d /content/

#NOTE: Update these TFRecord names to your files containing training set!
#Also, Update relevant rows:in training config file "ssd_mobilenet_v2_320x320_coco17_tpu-8.config"
#label_map_path,input_path

test_record_fname = '/content/mobilenet-transfer-learning/ThiefTFData/test/Thief.tfrecord'
train_record_fname = '/content/mobilenet-transfer-learning/ThiefTFData/train/Thief.tfrecord'
label_map_pbtxt_fname = '/content/mobilenet-transfer-learning/ThiefTFData/train/Thief_label_map.pbtxt'

/content
Cloning into 'mobilenet-transfer-learning'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 16 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (16/16), 24.15 MiB | 16.66 MiB/s, done.
/content/mobilenet-transfer-learning/ThiefTFData


## Step 6: Set configuration parameters for training the model:

In [2]:
#You can change chosen model to deploy different models available in the TF2 object detection zoo
MODELS_CONFIG = {
    'ssd_mobilenet_v2_320x320_coco17': {
        'model_name': 'ssd_mobilenet_v2_320x320_coco17_tpu-8',
        'base_pipeline_file': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz',
        'batch_size': 16
    }
}
chosen_model = 'ssd_mobilenet_v2_320x320_coco17'
num_steps = 1800 #40000 #The more steps, the longer the training. Increase if your loss function is still decreasing and validation metrics are increasing.
num_eval_steps = 500 #Perform evaluation after so many steps
model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
batch_size = MODELS_CONFIG[chosen_model]['batch_size'] #if you can fit a large batch in memory, it may speed up your trainin#g

## Step 7: Load trained model weights from TensorFlow site, and the custom configuration file which I’ve prepared for this example:

In [8]:
#Download pretrained weights
%mkdir /content/deploy/
%cd /content/deploy/
import tarfile
download_tar = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/' + pretrained_checkpoint

!wget {download_tar}
tar = tarfile.open(pretrained_checkpoint)
tar.extractall()
tar.close()
#Shorten the folder name,because long file paths are not yet supported :(
os.rename('ssd_mobilenet_v2_320x320_coco17_tpu-8','mobilnetv2')

/content/deploy
--2023-11-30 14:56:24--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.251.163.207, 142.251.167.207, 172.253.62.207, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|142.251.163.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46042990 (44M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz’

ssd_mobilenet_v2_32 100%[===================>]  43.91M   220MB/s    in 0.2s    

2023-11-30 14:56:24 (220 MB/s) - ‘ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz’ saved [46042990/46042990]



In [ ]:
%cd /content/deploy
download_config = '!wget https://raw.githubusercontent.com/tanhhkhcndn/mobilenet-transfer-learning/main/ssd_mobilenet_v2_320x320_coco17_tpu-8.config'
!wget {download_config}

In [11]:
#Prepare frozen model for retraining
fine_tune_checkpoint = '/content/deploy/mobilnetv2/checkpoint/ckpt-0'
pipeline_file = '/content/deploy/ssd_mobilenet_v2_320x320_coco17_tpu-8.config'
model_dir = '/content/training/'


def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())
num_classes = get_num_classes(label_map_pbtxt_fname)

ModuleNotFoundError: ignored